In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import multiprocessing as mp
import pickle
import time
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         FindResourceOrThrow, IrisInConfigurationSpace,
                         IrisOptions, JointSliders, LoadModelDirectives,
                         MeshcatVisualizer, MeshcatVisualizerParams, Parser,
                         ProcessModelDirectives,
                         RigidTransform, RollPitchYaw, Role, StartMeshcat)

from reproduction.util import *
from gcs.bezier import BezierGCS, BezierTrajectory
from IPython.display import HTML, SVG

from pydrake.all import (InverseDynamicsController, MultibodyPlant, Multiplexer, Demultiplexer,
                        StateInterpolatorWithDiscreteDerivative, TrajectorySource, Simulator,
                        ConstantVectorSource, Quaternion, MosekSolver, TrajectorySource,
                        MultibodyPositionToGeometryPose, SceneGraph)
from pydrake.all import LeafSystem, BsplineTrajectory, PiecewisePolynomial


from pydrake.common import configure_logging
configure_logging()

In [ ]:
meshcat = StartMeshcat()

In [ ]:
seed_points = {
    'nominal': [1.2, -0.8, 1.2, -2.1, -1.8, -0.1, -1.3, -2.4, -1.1, -1.3, 1.6, 0],
    'left_grasp': [1.65, -0.96, 1.6, -2.15, -1.58, 0.11, -1.3, -2.4, -1.1, -1.3, 1.6, 0],
    'hand_off': [1.53, -0.95, 1.6, -2.15, -3.1, 0., -1.52, -2.16, -1.66, -1., 3.1, 0.],
    'right_grasp': [1.2, -0.8, 1.2, -2.1, -1.8, -0.1, -1.75, -2.1, -1.85, -0.75, 1.6, -0.23],
}
X_sugar_start = RigidTransform(RollPitchYaw(-np.pi/2, 0, 0), [0.36, 0.36, 0.11])
X_sugar_left = RigidTransform(RollPitchYaw(0, -np.pi/2, 0), [0.0, 0.055, 0.0])
X_sugar_right = RigidTransform(RollPitchYaw(0, -np.pi/2, np.pi), [0.0, 0.043465316531254, 0.0])
X_sugar_end = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), [0.3315568024439854, -0.3841278846640738, 0.12])

def parse_homecart(plant):
    parser = Parser(plant)
    parser.package_map().Add("gcs", GcsDir())
    directives = LoadModelDirectives(
        FindModelFile('models/homecart.yaml'))
    ProcessModelDirectives(directives, plant, parser)

def parse_sugarbox_scenario(plant):
    parse_homecart(plant)
    Parser(plant).AddModelFromFile(
        FindResourceOrThrow(
            'drake/manipulation/models/ycb/sdf/004_sugar_box.sdf'))
    X_sugar = RigidTransform(RollPitchYaw(-np.pi/2, 0, 0), [0.36, 0.36, 0.11])
    plant.WeldFrames(plant.world_frame(),
                     plant.GetFrameByName('base_link_sugar'), X_sugar)


# Teleop to generate good seeds

In [ ]:
def teleop_seeds(q0=None):
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    meshcat.SetProperty('/Background', 'visible', False)
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
#     parse_sugarbox_scenario(plant)
    parse_homecart(plant)
    Parser(plant).AddModelFromFile(
       FindResourceOrThrow(
           'drake/manipulation/models/ycb/sdf/004_sugar_box.sdf'))
    plant.WeldFrames(plant.GetFrameByName('grasp_frame', plant.GetModelInstanceByName("gripper_left")),
                     plant.GetFrameByName('base_link_sugar'), X_sugar_left)
    plant.Finalize()
    
    meshcat_params = MeshcatVisualizerParams()
#     meshcat_params.delete_on_initialization_event = False
    meshcat_params.role = Role.kIllustration
#     meshcat_params.role = Role.kProximity
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, meshcat_params)

    sliders = builder.AddSystem(JointSliders(meshcat, plant, initial_value=q0))
    diagram = builder.Build()
    sliders.Run(diagram)
    # Print current value
    context = sliders.CreateDefaultContext()
    q = sliders.get_output_port().Eval(context)
    meshcat.DeleteAddedControls()
    print(f"q = {q}")
#     d_context = diagram.CreateDefaultContext()
#     sg_context = scene_graph.GetMyContextFromRoot(d_context)
#     plant_context = plant.GetMyMutableContextFromRoot(d_context)
#     plant.SetPositions(plant_context, q)
#     sugar_frame = plant.GetFrameByName('base_link_sugar', plant.GetModelInstanceByName("sugar_box_2"))
#     l_grip_frame = plant.GetFrameByName('grasp_frame', plant.GetModelInstanceByName("gripper_right"))
#     print(sugar_frame.CalcPose(plant_context, plant.world_frame()))
    return q

teleop_seeds(seed_points['nominal'])

In [ ]:
def ik_seeds(seed_points['nominal']):
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    meshcat.SetProperty('/Background', 'visible', False)
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parse_homecart(plant)
    plant.Finalize()



In [ ]:
def run_iris(seed):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parse_homecart(plant)
    plant.Finalize()
    diagram = builder.Build()

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    plant.SetPositions(plant_context, seed)
    
    iris_options = IrisOptions()
    iris_options.require_sample_point_is_contained = True
    iris_options.iteration_limit = 5
    iris_options.termination_threshold = -1
    iris_options.relative_termination_threshold = 0.02
    iris_options.enable_ibex = False

    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, iris_options)
    print("Time:", np.round((time.time() - start_time)/60., 4),
          "minutes.\tFaces", len(hpoly.b()), flush=True)
    return hpoly

run_iris(seed_points['nominal'])

# Iris region generation

In [ ]:
iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.02
iris_options.enable_ibex = False
iris_options.configuration_space_margin = 0.005

In [ ]:
for X, name in zip([X_sugar_start, X_sugar_left, X_sugar_right, X_sugar_end], ["start", "left", "right", "end"]):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parse_homecart(plant)
    Parser(plant).AddModelFromFile(
       FindResourceOrThrow(
           'drake/manipulation/models/ycb/sdf/004_sugar_box.sdf'))
    base_frame = plant.world_frame()
    if name == "left":
        base_frame = plant.GetFrameByName('grasp_frame', plant.GetModelInstanceByName("gripper_left"))
    elif name == "right":
        base_frame = plant.GetFrameByName('grasp_frame', plant.GetModelInstanceByName("gripper_right"))
    plant.WeldFrames(base_frame, plant.GetFrameByName('base_link_sugar'), X)
    plant.Finalize()
    diagram = builder.Build()

    def calcRegion(seed):
        start_time = time.time()
        context = diagram.CreateDefaultContext()
        plant_context = plant.GetMyContextFromRoot(context)
        plant.SetPositions(plant_context, seed)
        try:
            hpoly = IrisInConfigurationSpace(plant, plant_context, iris_options)
            print("Seed:", seed, "\tTime:", time.time() - start_time, flush=True)
            return hpoly
        except:
            print("Seed:", seed, "Failed.")
            return None

    def generateRegions(seed_points):
        seeds = list(seed_points.values()) if type(seed_points) is dict else seed_points
        regions = []
        loop_time = time.time()
        with mp.Pool(processes = 4) as pool:
            regions = pool.starmap(calcRegion, [[seed] for seed in seeds])

        print("Loop time:", time.time() - loop_time)

        if type(seed_points) is dict:
            return dict(list(zip(seed_points.keys(), regions)))
        
        for key in regions:
            if regions[key] is None:
                regions.pop(key)

        return regions

    print("Starting region generation for", name)
    regions = generateRegions(seed_points)

    with open(os.path.join(GcsDir(), 'data/homecart/iris_regions_' + name + '.reg'), 'wb') as f:
        pickle.dump(regions,f)

In [ ]:
from gcs.bezier import BezierGCS
from IPython.display import HTML, SVG

# for name in [None, "start", "left", "right", "end"]:
name = "end"
    
filename = os.path.join(GcsDir(), 'data/homecart/iris_regions.reg')
if name is not None:
    filename = os.path.join(GcsDir(), 'data/homecart/iris_regions_' + name + '.reg')

with open(filename, "rb") as f:
    regions = pickle.load(f)

for key in regions:
    print(key, regions[key].A().shape)

gcs = BezierGCS(regions, 3, 1)
SVG(gcs.VisualizeGraph())

# Generate GCS and plan trajectories

In [ ]:
gcs_dict = {}
for name in ["start", "left", "right", "end"]:
    filename = os.path.join(GcsDir(), 'data/homecart/iris_regions_' + name + '.reg')

    with open(filename, "rb") as f:
        regions = pickle.load(f)

    gcs_dict[name] = BezierGCS(regions, 4, 2)
    gcs_dict[name].addTimeCost(1)
    gcs_dict[name].addPathLengthCost(1)
    gcs_dict[name].addDerivativeRegularization(1e-3, 1e-3, 2)
    gcs_dict[name].addVelocityLimits(np.full(12, -1.), np.full(12, 1.))
    gcs_dict[name].setPaperSolverOptions()
    gcs_dict[name].setSolver(MosekSolver())

In [ ]:
start_time = time.time()
gcs_dict["start"].addSourceTarget(seed_points["nominal"], seed_points["left_grasp"], zero_deriv_boundary=2)
traj_1 = gcs_dict["start"].SolvePath(True, False, preprocessing=True)[0]
gcs_dict["start"].ResetGraph()
print(time.time() - start_time)

start_time = time.time()
gcs_dict["left"].addSourceTarget(seed_points["left_grasp"], seed_points["hand_off"], zero_deriv_boundary=2)
traj_2 = gcs_dict["left"].SolvePath(True, False, preprocessing=True)[0]
gcs_dict["left"].ResetGraph()
print(time.time() - start_time)

start_time = time.time()
gcs_dict["right"].addSourceTarget(seed_points["hand_off"], seed_points["right_grasp"], zero_deriv_boundary=2)
traj_3 = gcs_dict["right"].SolvePath(True, False, preprocessing=True)[0]
gcs_dict["right"].ResetGraph()
print(time.time() - start_time)

start_time = time.time()
gcs_dict["end"].addSourceTarget(seed_points["right_grasp"], seed_points["nominal"], zero_deriv_boundary=2)
traj_4 = gcs_dict["end"].SolvePath(True, False, preprocessing=True)[0]
gcs_dict["end"].ResetGraph()
print(time.time() - start_time)

traj_vec = [traj_1, traj_2, traj_3, traj_4]

# Convert trajectories to PiecewisePolynomials and save

In [ ]:
new_traj_vec = []
for traj in traj_vec:
    breaks = np.linspace(0, traj.end_time(), traj.end_time() * 1e3)
    knots = traj.vector_values(breaks)
    new_traj_vec.append(PiecewisePolynomial.CubicWithContinuousSecondDerivatives(
        breaks, knots, np.zeros(12), np.zeros(12)))

In [ ]:
for ii in range(len(new_traj_vec)):
    traj = new_traj_vec[ii]
    breaks = traj.get_segment_times()
    knots = traj.vector_values(breaks)
    np.savez(os.path.join(GcsDir(), "data/homecart/traj_" + str(ii) + "_data.npz"), breaks=breaks, knots=knots)

# Load saved PiecewisePolynomial trajectories

In [ ]:
new_traj_vec = []
for ii in range(4):
    data = np.load(os.path.join(GcsDir(), "data/homecart/traj_" + str(ii) + "_data.npz"))
    new_traj_vec.append(PiecewisePolynomial.CubicWithContinuousSecondDerivatives(
        data["breaks"], data["knots"], np.zeros(12), np.zeros(12)))

In [ ]:
from pydrake.all import RevoluteJoint
import yaml

plant = MultibodyPlant(time_step=0.001)
parse_homecart(plant)
plant.Finalize()

joint_list = []
for joint_index in plant.GetJointIndices(plant.GetModelInstanceByName("ur3_left")):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint_list.append("ur3_left::" + joint.name())
for joint_index in plant.GetJointIndices(plant.GetModelInstanceByName("ur3_right")):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint_list.append("ur3_right::" + joint.name())


for ii in range(4):
    data = np.load(os.path.join(GcsDir(), "data/homecart/traj_" + str(ii) + "_data.npz"))
#     waypoints = []
#     for jj in range(data["knots"].shape[1]):
#         waypoints.append(data["knots"][:, jj])
    waypoints = data["knots"].T.tolist()
        
    traj_data = {"joint_names": joint_list, "position_waypoints": waypoints}
    
    with open(os.path.join(GcsDir(), "data/homecart/traj_" + str(ii) + "_waypoints_lists.yaml"), 'w') as outfile:
        yaml.dump(traj_data, outfile, default_flow_style=False)
        outfile.flush()

# Visualize trajectories

In [ ]:
class HomecartTrajectorySource(LeafSystem):
    def __init__(self, trajectories):
        LeafSystem.__init__(self)
        assert len(trajectories) == 4
        self.trajectories = []
        for ii in range(4):
            gripper_set_point = [-0.107/2, 0.107/2, -0.107/2, 0.107/2]
            if ii == 1:
                # Add L close trajectory
                self.trajectories.append(PiecewisePolynomial.FirstOrderHold(
                    [0, 1], np.vstack((np.hstack((seed_points["left_grasp"][:6], [-0.107/2, 0.107/2],
                                                  seed_points["left_grasp"][6:], [-0.107/2, 0.107/2])),
                                       np.hstack((seed_points["left_grasp"][:6], [-0.035/2, 0.035/2],
                                                  seed_points["left_grasp"][6:], [-0.107/2, 0.107/2])))).T))
                gripper_set_point = [-0.035/2, 0.035/2, -0.107/2, 0.107/2]
            elif ii == 2:
                # Add R close trajectory
                self.trajectories.append(PiecewisePolynomial.FirstOrderHold(
                    [0, 1], np.vstack((np.hstack((seed_points["hand_off"][:6], [-0.035/2, 0.035/2],
                                                  seed_points["hand_off"][6:], [-0.107/2, 0.107/2])),
                                       np.hstack((seed_points["hand_off"][:6], [-0.035/2, 0.035/2],
                                                  seed_points["hand_off"][6:], [-0.035/2, 0.035/2])))).T))
                # Add L open trajectory
                self.trajectories.append(PiecewisePolynomial.FirstOrderHold(
                    [0, 1], np.vstack((np.hstack((seed_points["hand_off"][:6], [-0.035/2, 0.035/2],
                                                  seed_points["hand_off"][6:], [-0.035/2, 0.035/2])),
                                       np.hstack((seed_points["hand_off"][:6], [-0.107/2, 0.107/2],
                                                  seed_points["hand_off"][6:], [-0.035/2, 0.035/2])))).T))
                gripper_set_point = [-0.107/2, 0.107/2, -0.035/2, 0.035/2]
            elif ii == 3:
                # Add R open trajectory
                self.trajectories.append(PiecewisePolynomial.FirstOrderHold(
                    [0, 1], np.vstack((np.hstack((seed_points["right_grasp"][:6], [-0.107/2, 0.107/2],
                                                  seed_points["right_grasp"][6:], [-0.035/2, 0.035/2])),
                                       np.hstack((seed_points["right_grasp"][:6], [-0.107/2, 0.107/2],
                                                  seed_points["right_grasp"][6:], [-0.107/2, 0.107/2])))).T))
            full_traj = trajectories[ii]
            full_path = []
            if isinstance(full_traj, BezierTrajectory):
                for point in full_traj.path_traj.control_points():
                    full_path.append(np.concatenate((point[:6], np.array([gripper_set_point]).T[:2],
                                                     point[6:], np.array([gripper_set_point]).T[2:])))
                self.trajectories.append(BezierTrajectory(
                    BsplineTrajectory(full_traj.path_traj.basis(), full_path), full_traj.time_traj))
            elif isinstance(full_traj, PiecewisePolynomial):
                for traj_t in full_traj.get_segment_times():
                    point = full_traj.value(traj_t)
                    full_path.append(np.concatenate((point[:6], np.array([gripper_set_point]).T[:2],
                                                     point[6:], np.array([gripper_set_point]).T[2:])))
                self.trajectories.append(PiecewisePolynomial.CubicWithContinuousSecondDerivatives(
                    full_traj.get_segment_times(), full_path, np.zeros(16), np.zeros(16)))
            else:
                print(type(full_traj))
                raise TypeError()
        
        
        self.start_time = [0]
        for traj in self.trajectories:
            self.start_time.append(self.start_time[-1] + traj.end_time())
        self.start_time = np.array(self.start_time)
        self.port = self.DeclareVectorOutputPort("traj_eval", 12+4+7, self.DoVecTrajEval, {self.time_ticket()})
        
        self.plant = MultibodyPlant(time_step=0.0)
        parse_homecart(self.plant)
        self.plant.Finalize()
        self.context = self.plant.CreateDefaultContext()
    
    def DoVecTrajEval(self, context, output):
        t = context.get_time()
        traj_index = np.argmax(self.start_time > t) - 1
        
        if t >= self.start_time[-1]:
            q = self.trajectories[-1].value(self.trajectories[-1].end_time())
            traj_index = len(self.trajectories)
        else: 
            q = self.trajectories[traj_index].value(t - self.start_time[traj_index])
        
        self.plant.SetPositions(self.context, np.concatenate((q[:6], q[8:14])))
        
        X_sugar = None
        base_frame = self.plant.world_frame()
        if traj_index < 2:
            X_sugar = X_sugar_start
        elif traj_index < 5:
            X_sugar = X_sugar_left
            base_frame = self.plant.GetFrameByName('grasp_frame', self.plant.GetModelInstanceByName("gripper_left"))
        elif traj_index < 7:
            X_sugar = X_sugar_right
            base_frame = self.plant.GetFrameByName('grasp_frame', self.plant.GetModelInstanceByName("gripper_right"))
        else:
            X_sugar = X_sugar_end
            
        X_SW = base_frame.CalcPose(self.context, self.plant.world_frame()).multiply(X_sugar)
        sugar_q = np.concatenate((Quaternion(X_SW.rotation().matrix()).wxyz(), X_SW.translation()))
        
        output.set_value(np.vstack((q, np.expand_dims(sugar_q, 1))))

In [ ]:
builder = DiagramBuilder()

scene_graph = builder.AddSystem(SceneGraph())
plant = MultibodyPlant(time_step=0.0)
plant.RegisterAsSourceForSceneGraph(scene_graph)

parser = Parser(plant)
parser.package_map().Add("gcs", GcsDir())
# directives = LoadModelDirectives(
#     FindModelFile('models/homecart.yaml'))
directives = LoadModelDirectives(
    FindResourceOrThrow('drake/manipulation/models/tri_homecart/homecart.yaml'))
ProcessModelDirectives(directives, plant, parser)

parser.AddModelFromFile(
   FindResourceOrThrow(
       'drake/manipulation/models/ycb/sdf/004_sugar_box.sdf'))
plant.Finalize()

to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

traj_system = builder.AddSystem(HomecartTrajectorySource(new_traj_vec))
builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())

meshcat_params = MeshcatVisualizerParams()
meshcat_params.delete_on_initialization_event = False
meshcat_cpp = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, meshcat_params)
meshcat.Delete()

diagram = builder.Build()

simulator = Simulator(diagram)
meshcat_cpp.StartRecording()
simulator.AdvanceTo(9)
meshcat_cpp.PublishRecording()

In [ ]:
with open (os.path.join(GcsDir(), "data/homecart/handoff.html"), "w") as f:
   f.write(meshcat.StaticHtml())